# ELMo 표현을 사용해서 스팸 메일 분류하기

In [1]:
%tensorflow_version 1.x 

UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
pip install tensorflow-hub

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np

In [19]:
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)

sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

RuntimeError: variable_scope module_4/ was unused but the corresponding name_scope was already taken.

In [10]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin-1')
data[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [11]:
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
y_data = list(data['v1'])
X_data = list(data['v2'])

In [12]:
X_data[:5]

[&#39;Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...&#39;,
 &#39;Ok lar... Joking wif u oni...&#39;,
 &quot;Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&amp;C&#39;s apply 08452810075over18&#39;s&quot;,
 &#39;U dun say so early hor... U c already then say...&#39;,
 &quot;Nah I don&#39;t think he goes to usf, he lives around here though&quot;]

In [13]:
print(y_data[:5])

[0, 0, 1, 0, 0]


In [15]:
print(len(X_data))
n_of_train = int(len(X_data)* 0.8)
n_of_test = int(len(X_data) - n_of_train)
print(n_of_train)
print(n_of_test)

5572
4457
1115


In [16]:
X_train = np.asarray(X_data[:n_of_train]) #X_data 데이터 중에서 앞의 4457개의 데이터만 저장
y_train = np.asarray(y_data[:n_of_train]) #y_data 데이터 중에서 앞의 4457개의 데이터만 저장
X_test = np.asarray(X_data[n_of_train:]) #X_data 데이터 중에서 뒤의 1115개의 데이터만 저장
y_test = np.asarray(y_data[n_of_train:]) #y_data 데이터 중에서 뒤의 1115개의 데이터만 저장

In [17]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict = True, signature = 'default')["default"]

In [18]:
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)
model = Model(inputs = [input_text], outputs = output_layer)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = 'accuracy')

NameError: name &#39;elmo&#39; is not defined

In [20]:
history = model.fit(X_train, y_train, epochs=1, batch_size=60)
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

NameError: name &#39;model&#39; is not defined